In [152]:
!pip -q install langchain langchain-community langchain-openai openai langchain_experimental langchain-groq gradio

In [153]:
import pandas as pd
import glob
import os
import ast
import openai
from openai import OpenAI
from langchain_core.prompts import PromptTemplate
import json
from io import StringIO
from langchain_groq import ChatGroq
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from google.colab import files
import numpy as np

In [154]:
folder_path='/content/data-csv'
file_paths = glob.glob(os.path.join(folder_path, '*.csv'))
file_paths

['/content/data-csv/Business5_accounts.csv',
 '/content/data-csv/Business4_accounts.csv']

In [155]:
def load_csv_content(file_path):
    """
    Load a CSV file and return its content as a string.

    :param file_path: Path to the CSV file.
    :return: CSV content as a string.
    """
    df = pd.read_csv(file_path)
    return df.to_csv(index=False)

def prepare_files_for_prompt(file_paths):
    """
    Prepare a list of files in a format suitable for passing to the OpenAI API.

    :param file_paths: List of file paths.
    :return: List of dictionaries, each containing the filename and content.
    """
    files = []
    for path in file_paths:
        content = load_csv_content(path)
        files.append({"filename": path.split("/")[-1], "content": content})
    return files

In [156]:
file_contents = prepare_files_for_prompt(file_paths)

In [157]:
file_contents

[{'filename': 'Business5_accounts.csv',
  'content': 'Outflow,Location,ClientName,AccountLead,Revenue,RecordDate,SurplusDeficit\n34381.66,East,Client_1,Manager_8,63024.57,14:17.2,1322.73\n35655.69,West,Client_2,Manager_10,84398.17,14:17.2,4588.24\n23537.13,West,Client_3,Manager_6,57527.18,14:17.2,-4093.33\n11968.86,South,Client_4,Manager_7,39567.06,14:17.2,346.26\n11577.83,South,Client_5,Manager_6,59923.99,14:17.2,4609.26\n43124.27,East,Client_6,Manager_5,86594.78,14:17.2,-4195.43\n31038.63,South,Client_7,Manager_5,75402.48,14:17.2,-403.61\n45686.65,East,Client_8,Manager_3,70821.35,14:17.2,2665.08\n19907.81,North,Client_9,Manager_1,32135.44,14:17.2,1477.93\n35130.92,North,Client_10,Manager_10,4131.19,14:17.2,-3233.82\n'},
 {'filename': 'Business4_accounts.csv',
  'content': 'Advisor,GrossIncome,DateOfTransaction,ClientName,Costs,NetResult,Zone,Owner\nManager_5,53582.61,14:17.2,Client_1,2215.81,-4510.81,North,Tata\nManager_4,45489.04,14:17.2,Client_2,14581.87,-3517.44,South,Birla\nManag

In [158]:
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI')
openai.api_key = os.environ['OPENAI_API_KEY']
client = OpenAI()

groq_api = 'YOUR GROQ API'
llm = ChatGroq(temperature=0, model="llama3-70b-8192", api_key=groq_api)

In [159]:
from pydantic import BaseModel
class Response(BaseModel):
  mapping:dict

In [170]:


def generate_mapping_response(files):
    response = client.chat.completions.create(
        model="gpt-4o-2024-08-06",
        messages=[
            {
                "role": "system",
                "content": "You are an expert in data processing and merging financial and accounts data from CSV files. Your task is to analyze the columns and generate a precise mapping."
            },
            {
                "role": "user",
                "content": """You are given several CSV files containing financial and accounts data. Your task is to:

                1. **Identify Similar Columns:** Analyze the columns in each CSV file by comparing both column names and data content. Ensure that columns with different names but similar data are mapped together.

                2. **Generate Column Mapping:** Create a mapping of similar columns across all CSV files. For columns that do not match, keep them separate in the final mapping with their original names.

                3. **Output:** Provide a object containing the mapping for each file. The format should be:
                   - { 'File-1': {'OriginalColumnName1': 'MappedColumnName', 'OriginalColumnName2': 'MappedColumnName'}, 'File-2': {'OriginalColumnName3': 'MappedColumnName', ...}, ... }

                ### Additional Instructions:
                - Ensure that similar columns are mapped accurately, even if their names differ.
                - Focus on correctly identifying and mapping financial and accounts-related columns.
                - No need to merge the files, just provide the mapping."""
            },
            {
                "role":"user",
                "content":"Give only the mapping as response,nothing else,response should an Dictionary with single quotes around it"
            },
            {
                "role":"user",
                "content":"Map every file to last file column names, and keep last file column names mapped to itself"
            },
            {
                "role": "user",
                "content": f"Here are the contents of the CSV files: {files}"
            }
        ],
        temperature=0
    )

    return response.choices[0].message.content


In [171]:
dict_str=generate_mapping_response(file_contents)

mapping = ast.literal_eval(dict_str)

In [172]:
mapping,file_paths

({'Business5_accounts.csv': {'Outflow': 'Costs',
   'Location': 'Zone',
   'ClientName': 'ClientName',
   'AccountLead': 'Advisor',
   'Revenue': 'GrossIncome',
   'RecordDate': 'DateOfTransaction',
   'SurplusDeficit': 'NetResult'},
  'Business4_accounts.csv': {'Advisor': 'Advisor',
   'GrossIncome': 'GrossIncome',
   'DateOfTransaction': 'DateOfTransaction',
   'ClientName': 'ClientName',
   'Costs': 'Costs',
   'NetResult': 'NetResult',
   'Zone': 'Zone',
   'Owner': 'Owner'}},
 ['/content/data-csv/Business5_accounts.csv',
  '/content/data-csv/Business4_accounts.csv'])

In [177]:
def rename_and_merge_csv_files(mapping,file_paths):
  renamed_dfs=[]

  all_columns =[]
  for file in file_paths:
    file_name=file[file.rindex('/')+1:]
    for column in mapping[file_name].values():
      if column not in all_columns:
        all_columns.append(column)

  for file in file_paths:
    file_name=file[file.rindex('/')+1:]
    df=pd.read_csv(file)
    df=df.rename(columns=mapping[file_name])
    for column in all_columns:
      if column not in df.columns:
        df[column] = np.nan
    renamed_dfs.append(df)

  return pd.concat(renamed_dfs, axis=0, ignore_index=True)



merged_df=rename_and_merge_csv_files(mapping,file_paths)

In [178]:
merged_df

,Costs,Zone,ClientName,Advisor,GrossIncome,DateOfTransaction,NetResult,Owner
0,34381.66,East,Client_1,Manager_8,63024.57,14:17.2,1322.73,NaN
1,35655.69,West,Client_2,Manager_10,84398.17,14:17.2,4588.24,NaN
2,23537.13,West,Client_3,Manager_6,57527.18,14:17.2,-4093.33,NaN
3,11968.86,South,Client_4,Manager_7,39567.06,14:17.2,346.26,NaN
4,11577.83,South,Client_5,Manager_6,59923.99,14:17.2,4609.26,NaN
5,43124.27,East,Client_6,Manager_5,86594.78,14:17.2,-4195.43,NaN
6,31038.63,South,Client_7,Manager_5,75402.48,14:17.2,-403.61,NaN
7,45686.65,East,Client_8,Manager_3,70821.35,14:17.2,2665.08,NaN
8,19907.81,North,Client_9,Manager_1,32135.44,14:17.2,1477.93,NaN
9,35130.92,North,Client_10,Manager_10,4131.19,14:17.2,-3233.82,NaN


In [183]:
def mkdir(merged_df):
  folder_path = '/content/normalized_data_folder'
  file_name = 'normalized_data.csv'
  file_path = os.path.join(folder_path, file_name)

  # Create the directory if it does not exist
  os.makedirs(folder_path, exist_ok=True)
  merged_df.to_csv(file_path, index=False)
  return file_path
file_path=mkdir(merged_df)

In [184]:
agent = create_csv_agent(llm,file_path, verbose=True, allow_dangerous_code=True)

In [186]:
def query_data(query):
    response = agent.invoke(query)
    return response

In [187]:
query = "total income and specifically mention which manager brought how much income"
response = query_data(query)
print(response['output'])



> Entering new AgentExecutor chain...
Thought: I need to calculate the total income and group it by Advisor to see which manager brought how much income.

Action: python_repl_ast
Action Input: `df['GrossIncome'].sum()`1208594.95I now know the total income, but I still need to group it by Advisor to see which manager brought how much income.

Action: python_repl_ast
Action Input: `df.groupby('Advisor')['GrossIncome'].sum()`Advisor
Manager_1     151721.38
Manager_10     88529.36
Manager_3      70821.35
Manager_4      45489.04
Manager_5     410368.46
Manager_6     117451.17
Manager_7      90656.88
Manager_8      63024.57
Manager_9     170532.74
Name: GrossIncome, dtype: float64I now know the final answer.

Final Answer: The total income is 1208594.95. The income brought by each manager is: Manager_1: 151721.38, Manager_10: 88529.36, Manager_3: 70821.35, Manager_4: 45489.04, Manager_5: 410368.46, Manager_6: 117451.17, Manager_7: 90656.88, Manager_8: 63024.57, Manager_9: 170532.74.

> Fin

In [ ]:
import gradio as gr


def processing(query):
        response = query_data(query)
        return response['output']

demo = gr.Interface(
    fn=processing,
    inputs=["text"],
    outputs=["text"],
)

demo.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://04d0be8338a492d1e0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)




> Entering new AgentExecutor chain...
Thought: I need to count the number of non-null values in the 'Owner' column for each zone.

Action: python_repl_ast
Action Input: `df['Owner'].notnull().groupby(df['Zone']).sum()`Zone
East     1
North    1
South    2
West     4
Name: Owner, dtype: int64I now know the final answer.

Final Answer: The zone with the highest count of owner present is West with 4 owners.

> Finished chain.
